Các bước xây dựng chương trình học máy với Pyspark:

Bước 1) Hoạt động cơ bản với PySpark

Bước 2) Tiền xử lý dữ liệu

Bước 3) Xây dựng pipeline xử lý dữ liệu

Bước 4) Xây dựng bộ phân loại: logistic

Bước 5) Đào tạo và đánh giá mô hình

Bước 6) Điều chỉnh siêu tham số

(Nguồn: https://tek4.vn/apache-spark-machine-learning-voi-pyspark-va-mllib/)

In [1]:
!pip install pyspark

     |████████████████████████████████| 212.3MB 62kB/s 
     |████████████████████████████████| 204kB 45.1MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=8a1bff29019318b4833eee718f95d3bf04ef44cfcaf3e7469c3e34db3ec4bb0f
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [3]:
import pyspark

from pyspark import SparkContext
sc =SparkContext()

In [6]:
from pyspark import SparkFiles
from pyspark.sql import Row
from pyspark.sql import SQLContext

sc.addFile('adult_data.csv')
sqlContext = SQLContext(sc)

In [10]:
columns = ['age','workclass','fnlwgt','education','education_num',
           'marital','occupation','relationship','race','sex','capital_gain','capital_loss',
           'hours_week','native_country','label']
           
df = sqlContext.read.csv(SparkFiles.get("adult_data.csv"), header=True, inferSchema= True).toDF(*columns)
df.show()

+---+-----------------+--------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+------------+------------+----------+--------------+------+
|age|        workclass|  fnlwgt|    education|education_num|             marital|        occupation|  relationship|               race|    sex|capital_gain|capital_loss|hours_week|native_country| label|
+---+-----------------+--------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+------------+------------+----------+--------------+------+
| 50| Self-emp-not-inc| 83311.0|    Bachelors|         13.0|  Married-civ-spouse|   Exec-managerial|       Husband|              White|   Male|         0.0|         0.0|      13.0| United-States| <=50K|
| 38|          Private|215646.0|      HS-grad|          9.0|            Divorced| Handlers-cleaners| Not-in-family|              White|   Male|         0.0|         0.0|      40.0| United-

In [11]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: double (nullable = true)
 |-- education: string (nullable = true)
 |-- education_num: double (nullable = true)
 |-- marital: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital_gain: double (nullable = true)
 |-- capital_loss: double (nullable = true)
 |-- hours_week: double (nullable = true)
 |-- native_country: string (nullable = true)
 |-- label: string (nullable = true)



In [12]:
df.select('age','fnlwgt').show(5)

+---+--------+
|age|  fnlwgt|
+---+--------+
| 50| 83311.0|
| 38|215646.0|
| 53|234721.0|
| 28|338409.0|
| 37|284582.0|
+---+--------+
only showing top 5 rows



In [13]:
df.crosstab('age', 'label').sort("age_label").show()

+---------+------+-----+
|age_label| <=50K| >50K|
+---------+------+-----+
|       17|   395|    0|
|       18|   550|    0|
|       19|   710|    2|
|       20|   753|    0|
|       21|   717|    3|
|       22|   752|   13|
|       23|   865|   12|
|       24|   767|   31|
|       25|   788|   53|
|       26|   722|   63|
|       27|   754|   81|
|       28|   748|  119|
|       29|   679|  134|
|       30|   690|  171|
|       31|   705|  183|
|       32|   639|  189|
|       33|   684|  191|
|       34|   643|  243|
|       35|   659|  217|
|       36|   635|  263|
+---------+------+-----+
only showing top 20 rows



In [15]:
df.groupBy("native_country").count().sort("count",ascending=True).show()

+--------------------+-----+
|      native_country|count|
+--------------------+-----+
|  Holand-Netherlands|    1|
|            Scotland|   12|
|            Honduras|   13|
|             Hungary|   13|
| Outlying-US(Guam...|   14|
|          Yugoslavia|   16|
|                Laos|   18|
|            Thailand|   18|
|            Cambodia|   19|
|     Trinadad&Tobago|   19|
|                Hong|   20|
|             Ireland|   24|
|             Ecuador|   28|
|              Greece|   29|
|              France|   29|
|                Peru|   31|
|           Nicaragua|   34|
|            Portugal|   37|
|                Iran|   43|
|               Haiti|   44|
+--------------------+-----+
only showing top 20 rows



In [17]:
df_remove = df.filter(df.native_country !=	'Holand-Netherlands')

In [18]:
df_remove.groupBy("native_country").count().sort("count",ascending=True).show()

+--------------------+-----+
|      native_country|count|
+--------------------+-----+
|  Holand-Netherlands|    1|
|            Scotland|   12|
|            Honduras|   13|
|             Hungary|   13|
| Outlying-US(Guam...|   14|
|          Yugoslavia|   16|
|                Laos|   18|
|            Thailand|   18|
|            Cambodia|   19|
|     Trinadad&Tobago|   19|
|                Hong|   20|
|             Ireland|   24|
|             Ecuador|   28|
|              Greece|   29|
|              France|   29|
|                Peru|   31|
|           Nicaragua|   34|
|            Portugal|   37|
|                Iran|   43|
|               Haiti|   44|
+--------------------+-----+
only showing top 20 rows



In [20]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
 
stringIndexer = StringIndexer(inputCol="workclass", outputCol="workclass_encoded")
model = stringIndexer.fit(df)
indexed = model.transform(df)


In [23]:
encoder = OneHotEncoder(dropLast=False, inputCol="workclass_encoded", outputCol="workclass_vec")
encode = encoder.fit(indexed)
encoded = encode.transform(indexed)
encoded.show(2)

+---+-----------------+--------+----------+-------------+-------------------+------------------+--------------+------+-----+------------+------------+----------+--------------+------+-----------------+-------------+
|age|        workclass|  fnlwgt| education|education_num|            marital|        occupation|  relationship|  race|  sex|capital_gain|capital_loss|hours_week|native_country| label|workclass_encoded|workclass_vec|
+---+-----------------+--------+----------+-------------+-------------------+------------------+--------------+------+-----+------------+------------+----------+--------------+------+-----------------+-------------+
| 50| Self-emp-not-inc| 83311.0| Bachelors|         13.0| Married-civ-spouse|   Exec-managerial|       Husband| White| Male|         0.0|         0.0|      13.0| United-States| <=50K|              1.0|(9,[1],[1.0])|
| 38|          Private|215646.0|   HS-grad|          9.0|           Divorced| Handlers-cleaners| Not-in-family| White| Male|         0.0

In [25]:
from pyspark.ml import Pipeline
#from pyspark.ml.feature import OneHotEncoderEstimator
CATE_FEATURES = ['workclass', 'education', 'marital', 'occupation', 'relationship', 'race', 'sex', 'native_country']
stages = [] # stages in our Pipeline
for categoricalCol in CATE_FEATURES:
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index")
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()],
                                     outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]

In [26]:
label_stringIdx =  StringIndexer(inputCol="label", outputCol="newlabel")
stages += [label_stringIdx]

In [31]:
CONTI_FEATURES  = ['age', 'fnlwgt','capital_gain', 'education_num', 'capital_loss', 'hours_week']
assemblerInputs = [c + "classVec" for c in CATE_FEATURES] + CONTI_FEATURES

assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [32]:
pipeline = Pipeline(stages=stages)
pipelineModel = pipeline.fit(df_remove)
model = pipelineModel.transform(df_remove)

In [33]:
model.take(1)

[Row(age=50, workclass=' Self-emp-not-inc', fnlwgt=83311.0, education=' Bachelors', education_num=13.0, marital=' Married-civ-spouse', occupation=' Exec-managerial', relationship=' Husband', race=' White', sex=' Male', capital_gain=0.0, capital_loss=0.0, hours_week=13.0, native_country=' United-States', label=' <=50K', workclassIndex=1.0, workclassclassVec=SparseVector(8, {1: 1.0}), educationIndex=2.0, educationclassVec=SparseVector(15, {2: 1.0}), maritalIndex=0.0, maritalclassVec=SparseVector(6, {0: 1.0}), occupationIndex=2.0, occupationclassVec=SparseVector(14, {2: 1.0}), relationshipIndex=0.0, relationshipclassVec=SparseVector(5, {0: 1.0}), raceIndex=0.0, raceclassVec=SparseVector(4, {0: 1.0}), sexIndex=0.0, sexclassVec=SparseVector(1, {0: 1.0}), native_countryIndex=0.0, native_countryclassVec=SparseVector(41, {0: 1.0}), newlabel=0.0, features=SparseVector(100, {1: 1.0, 10: 1.0, 23: 1.0, 31: 1.0, 43: 1.0, 48: 1.0, 52: 1.0, 53: 1.0, 94: 50.0, 95: 83311.0, 97: 13.0, 99: 13.0}))]

In [34]:
from pyspark.ml.linalg import DenseVector
input_data = model.rdd.map(lambda x: (x["newlabel"], DenseVector(x["features"])))

In [36]:
df_train = sqlContext.createDataFrame(input_data, ["label", "features"])
df_train.show(2)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|[0.0,1.0,0.0,0.0,...|
|  0.0|[1.0,0.0,0.0,0.0,...|
+-----+--------------------+
only showing top 2 rows



In [37]:
train_data, test_data = df_train.randomSplit([.8,.2],seed=1234)

In [38]:
train_data.groupby('label').agg({'label': 'count'}).show()

+-----+------------+
|label|count(label)|
+-----+------------+
|  0.0|       19785|
|  1.0|        6254|
+-----+------------+



In [39]:
test_data.groupby('label').agg({'label': 'count'}).show()

+-----+------------+
|label|count(label)|
+-----+------------+
|  0.0|        4934|
|  1.0|        1587|
+-----+------------+



In [40]:
from pyspark.ml.classification import LogisticRegression
 
lr = LogisticRegression(labelCol="label",
                        featuresCol="features",
                        maxIter=10,
                        regParam=0.3)
 
linearModel = lr.fit(train_data)

In [41]:
print("Coefficients: " + str(linearModel.coefficients))
print("Intercept: " + str(linearModel.intercept))

Coefficients: [-0.07542478256679226,-0.13162170271883578,-0.08266122771952229,-0.1749616630960082,-0.1225200175313613,0.1735386141188351,0.19183390036686707,-0.5960323812296133,-0.19510929481849007,-0.07991255149987571,0.22180544183633938,0.38150058777063783,0.013472390879031692,-0.3089542597723308,-0.05618017014068094,-0.3139189746835882,-0.39343567591904477,0.5113988129963153,-0.39049236805543885,-0.19172648809986986,0.6223810841185622,-0.3729301825869573,-0.43626907947137,0.31825882153468105,-0.3429071838598613,-0.2079274185896027,-0.2232537703636267,-0.15657173728055793,-0.18063592243623472,0.1818242255607749,-0.07101502657034851,0.27894989565010986,-0.1176907570518571,0.055180203614051376,-0.2850086970767526,-0.21266668350164447,-0.17636013867069683,-0.1327475000836963,-0.2843479021127774,-0.33714235631660044,0.13091492300078966,0.12282793981440418,-0.2948259679140343,0.2631050045489643,-0.191004168732102,-0.3055050088308,-0.2512822831911514,0.41729199977762554,-0.0693409198966882

In [42]:
predictions = linearModel.transform(test_data)

In [43]:
selected = predictions.select("label", "prediction", "probability")
selected.show(20)

+-----+----------+--------------------+
|label|prediction|         probability|
+-----+----------+--------------------+
|  0.0|       0.0|[0.92873515300161...|
|  0.0|       0.0|[0.90088323495175...|
|  0.0|       0.0|[0.91164867507150...|
|  0.0|       0.0|[0.92343002123400...|
|  0.0|       0.0|[0.90623021836235...|
|  0.0|       0.0|[0.64427091454062...|
|  0.0|       0.0|[0.63546004973447...|
|  0.0|       0.0|[0.89108307408845...|
|  0.0|       0.0|[0.84236043482192...|
|  0.0|       0.0|[0.83155277070207...|
|  0.0|       0.0|[0.85176316154221...|
|  0.0|       0.0|[0.87451511061045...|
|  0.0|       0.0|[0.83330369453748...|
|  0.0|       0.0|[0.83104539496232...|
|  0.0|       0.0|[0.81831107902547...|
|  0.0|       0.0|[0.56304862961456...|
|  0.0|       0.0|[0.58214346857226...|
|  0.0|       0.0|[0.65689587400529...|
|  0.0|       0.0|[0.54334693843335...|
|  0.0|       0.0|[0.93121734948562...|
+-----+----------+--------------------+
only showing top 20 rows



In [44]:
cm = predictions.select("label", "prediction")
cm.groupby('label').agg({'label': 'count'}).show()

+-----+------------+
|label|count(label)|
+-----+------------+
|  0.0|        4934|
|  1.0|        1587|
+-----+------------+



In [45]:
cm.groupby('prediction').agg({'prediction': 'count'}).show()

+----------+-----------------+
|prediction|count(prediction)|
+----------+-----------------+
|       0.0|             5930|
|       1.0|              591|
+----------+-----------------+



In [46]:
cm.filter(cm.label == cm.prediction).count() / cm.count()

0.821806471400092

In [47]:
def accuracy_m(model): 
    predictions = model.transform(test_data)
    cm = predictions.select("label", "prediction")
    acc = cm.filter(cm.label == cm.prediction).count() / cm.count()
    print("Model accuracy: %.3f%%" % (acc * 100)) 
accuracy_m(model = linearModel)

Model accuracy: 82.181%


In [48]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
print(evaluator.evaluate(predictions))
print(evaluator.getMetricName())

0.8957906623255631
areaUnderROC


In [53]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
 
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.01, 0.5])
             .build())

In [54]:
from time import *
start_time = time()
 
cv = CrossValidator(estimator=lr,
                    estimatorParamMaps=paramGrid,
                    evaluator=evaluator, numFolds=5)

cvModel = cv.fit(train_data)

end_time = time() 
elapsed_time = end_time - start_time
print("Time to train model: %.3f seconds" % elapsed_time)

Time to train model: 836.011 seconds


In [55]:
accuracy_m(model = cvModel)

Model accuracy: 85.002%


In [56]:
bestModel = cvModel.bestModel
bestModel.extractParamMap()

{Param(parent='LogisticRegression_8a9c476e6a60', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2).'): 2,
 Param(parent='LogisticRegression_8a9c476e6a60', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0,
 Param(parent='LogisticRegression_8a9c476e6a60', name='family', doc='The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial'): 'auto',
 Param(parent='LogisticRegression_8a9c476e6a60', name='featuresCol', doc='features column name.'): 'features',
 Param(parent='LogisticRegression_8a9c476e6a60', name='fitIntercept', doc='whether to fit an intercept term.'): True,
 Param(parent='LogisticRegression_8a9c476e6a60', name='labelCol', doc='label column name.'): 'label',
 Param(parent='LogisticRegression_8a9c476e6a60', name='maxBlockSizeInMB', doc='maximum memory in MB for s